In [24]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from collections import namedtuple
import numpy as np

In [25]:
boston = load_boston()
X_train, X_test, Y_train, Y_test = train_test_split(boston.data, boston.target, test_size = 0.25)

In [26]:
class DecisionTree:
    def __init__(self, max_depth=5):
        self.root = None
        self.max_depth = max_depth
    
    class Node:
        def __init__(self, sample, feature, threshold, left_child, right_child):
            self.data = np.mean(sample)
            self.left_child = left_child
            self.right_child = right_child
            self.feature = feature
            self.threshold = threshold
    
    def get_part_condition(self, X, Y):
        min_crit = None
        part_condition = None
        for feature in range(len(X[0])):
            for threshold in X[:, feature]:
                left = Y[X[:, feature] < threshold]
                right = Y[X[:,feature] >= threshold]
                crit = len(left) * np.var(left)/len(Y) + len(right) * np.var(right)/len(Y)
                if min_crit > crit or min_crit is None:
                    min_crit = crit
                    part_condition = (feature, threshold)
                
        return part_condition[0], part_condition[1]
     
    def split(self, X, Y, feature, threshold):
        left = X[:, feature] < threshold
        right = X[:, feature] >= threshold
        
        return X[left], X[right], Y[left], Y[right]
    
    def get_partition(self, X, Y, depth):
        if depth == self.max_depth or np.var(Y) == 0 or len(X) <= 1:
            return self.Node(Y, None, None, None, None)
        feature, threshold = self.get_part_condition(X, Y)
        X_left, X_right, Y_left, Y_right = self.split(X, Y, feature, threshold)
        left_child = self.get_partition(X_left, Y_left, depth + 1)
        right_child = self.get_partition(X_right, Y_right, depth + 1)
        
        return self.Node(Y, feature, threshold, left_child, right_child)
    
    def fit(self, X, Y):
        self.root = self.get_partition(X, Y, 0)
        
    def node_predict(self, X, curr_node):
        if curr_node.feature is None or curr_node.threshold is None:
            return curr_node.data
        
        
        X_left, X_right, left, right = self.split(X, np.arange(len(X)), curr_node.feature, curr_node.threshold)
        Y = np.empty(len(X))
        Y[left] = self.node_predict(X_left, curr_node.left_child)
        Y[right] = self.node_predict(X_right, curr_node.right_child)
        return Y
    
    def predict(self, X):
        return self.node_predict(X, self.root)   

In [27]:
dec_tree = DecisionTree()
dec_tree.fit(X_train, Y_train)
dec_tree.predict(X_test[:10])

array([ 12.69772727,  20.99741379,  17.11489362,  17.11489362,
        23.89393939,  45.61428571,  20.99741379,  20.99741379,
        21.50909091,  20.99741379])

In [28]:
mean_squared_error(Y_test, dec_tree.predict(X_test))

28.068488368376538

In [29]:
mean_squared_error(Y_train, dec_tree.predict(X_train))

6.5466477634605438